<a href="https://colab.research.google.com/github/danieltwh/Sentiment-Analysis-Stock-Performance/blob/yilin/notebooks/TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mount google drive to colab

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import pandas as pd
import os

os.chdir("/content/drive/MyDrive/Fintech_Sentiment_Analysis_Personal/experiments/")

file = "labelled_marketaux_news_combined_2022-10-15.csv"
news_data = pd.read_csv(file)[['Title', '6m']].dropna()
print(news_data)
news_list = news_data['Title'].tolist()

                                                   Title   6m
0      Biotech : comment investir sur les biotechnolo...  0.0
1      Agilent Technologies : LC/MS Instruments Recei...  0.0
2      Clinical Oncology Next-generation Sequencing M...  0.0
3      Worldwide Endocrine Testing Industry To 2026 -...  0.0
4        InMode Ltd. : De retour sur un niveau important  0.0
...                                                  ...  ...
57910  Top investing minds are split over whether the...  1.0
57911        Tencent, Alibaba, Didi : Pékin serre la vis  1.0
57912  О начале торгов ценными бумагами 2 августа 202...  1.0
57913  Why New Oriental Education, TAL Education, and...  1.0
57914  TAL Education Group (TAL): Price Now Near $75....  1.0

[55032 rows x 2 columns]


In [ ]:
import numpy as np 
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import nltk
import re
import string
from nltk.stem import WordNetLemmatizer

nltk.download("stopwords")
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
X = news_data['Title']
y = news_data['6m']

# split train and test data    
from sklearn.model_selection import train_test_split
X_train_non, X_test_non, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

train_X = []

#text pre processing
for i in range(0, len(X_train_non)):
    review = re.sub('[^a-zA-Z]', ' ', X_train_non.iloc[i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    review = ' '.join(review)
    train_X.append(review)
print(X_train_non)
print(train_X)

31416    Top Gold & Silver Stocks to Watch for in May 2021
11222    Associated Banc-Corp Announces Redemption of A...
30738    Global Hyperspectral Imaging in Agriculture Ma...
31302    Hewlett Packard Enterprise Company's (HPE) Man...
21766    Mckinley Capital Management Llc Buys Casey's G...
                               ...                        
48774                       Sylebra Capital Ltd Buys PureC
55299    PropertyGuru appoints Rachna Bhasin to Board o...
45496                  Portefeuille d'actions de Ledep p.9
46450    Old Dominion Freight Line, Inc. (ODFL): Hedge ...
4055     Square Makes a Deal With Jay-Z, and Okta Buys ...
Name: Title, Length: 44025, dtype: object
['top gold silver stock watch may', 'associated banc corp announces redemption outstanding depositary share representing interest perpetual preferred stock series', 'global hyperspectral imaging agriculture market increased emphasis precision farming sustainable development', 'hewlett packard enterprise comp

In [ ]:
test_X = []

#text pre processing
for i in range(0, len(X_test_non)):
    review = re.sub('[^a-zA-Z]', ' ', X_test_non.iloc[i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
    review = ' '.join(review)
    test_X.append(review)
print(test_X)
print(X_test_non)

['gentex gntx report next week wall street expects earnings growth', 'stock hit week high friday', 'beleagured lordstown motor may showing green shoot use caution', 'fair isaac two fico leader win woman credit award', 'dynatrace pullback set stage gain', 'cardiopulmonary bypass equipment market expected grow cagr reach usd million according delveinsight', 'dzieje si tu teraz ryzyko wykluczenia du ej grupy pracownik w coraz wi ksze', 'stock hit week high wednesday', 'global digital signature market reach thousand', 'cgi help nexelis gain competitive advantage digitization business workflow', 'futu holding limited futu q earnings call transcript', 'wall street l euphorie ne retombe pa le bond flambent', 'aua capital management llc buy ishares short term national muni bond etf vanguard total world stock etf signet jeweler sell well fargo philip morris international inc exxon mobil corp', 'redwood investment llc buy', 'remarkably cheap canadian stock buy today', 'liberty star equity fund j

In [ ]:
#tf idf
tf_idf = TfidfVectorizer()
#applying tf idf to training data
X_train_tf = tf_idf.fit_transform(train_X)
#applying tf idf to training data
X_train_tf = tf_idf.transform(train_X)
print(X_train_tf)

print("n_samples: %d, n_features: %d" % X_train_tf.shape)

#transforming test data into tf-idf matrix
X_test_tf = tf_idf.transform(test_X)
print(X_test_tf)

print("n_samples: %d, n_features: %d" % X_test_tf.shape)


  (0, 23947)	0.463242688357973
  (0, 22419)	0.328810644208195
  (0, 21249)	0.1982822025737158
  (0, 20338)	0.5141147542714748
  (0, 13729)	0.45595113914062246
  (0, 9586)	0.40714680182648527
  (1, 21249)	0.09918422594830967
  (1, 20115)	0.16534179575863794
  (1, 20024)	0.2480760222227908
  (1, 18669)	0.3648311268015851
  (1, 18348)	0.3012275298048901
  (1, 17288)	0.2429941314445753
  (1, 16639)	0.339751367230848
  (1, 16051)	0.3012275298048901
  (1, 11437)	0.25754029789202637
  (1, 6087)	0.3648311268015851
  (1, 5170)	0.1318396877128985
  (1, 2212)	0.29547218027828215
  (1, 1690)	0.29241268725695846
  (1, 1186)	0.15036116111416709
  (2, 21612)	0.26964860057271595
  (2, 17269)	0.33263876207329457
  (2, 13602)	0.12568982205897408
  (2, 11094)	0.2834554710595681
  (2, 10935)	0.3265367044135607
  :	:
  (44021, 11115)	0.32737977770407256
  (44021, 6372)	0.45523246560361247
  (44021, 2919)	0.2224909808164675
  (44021, 2611)	0.4360302995178546
  (44021, 1356)	0.2563520637196514
  (44022, 1713

In [ ]:
# logistic regression

from sklearn.linear_model import LogisticRegression
from sklearn import metrics

logreg = LogisticRegression()
logreg.fit(X_train_tf, y_train)
y_pred = logreg.predict(X_test_tf)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy of logistic regression classifier on test set: 0.50


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [ ]:
# Random forest

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score

classifier = RandomForestClassifier(n_estimators=20, random_state=0)
classifier.fit(X_train_tf, y_train)
y_pred = classifier.predict(X_test_tf)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 48.41%
Mean Absolute Error: 0.6395021350049969
Mean Squared Error: 0.886799309530299
Root Mean Squared Error: 0.9417002227515394


In [ ]:
# XGBoost
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train_tf, y_train)
y_pred = model.predict(X_test_tf)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))